In [ ]:
import json
from mstr_robotics.redis_db import  redis_bi_analysis, fetch_it_all

redis_env_d={"mstr_dev":{
        "host":"redis-14995.c56.east-us.azure.redns.redis-cloud.com",
        "port":14995,
        "password":"KSvQA45EE8ZEJ3xcFvus96927CW56D8u",  
        "username":"default",  
        "decode_responses":True,
        "project_id":"B7CA92F04B9FAE8D941C3E9B7E0CD754",
        "redis_prefix":"mstr_dev"
},
        "mstr_test":{
        "host":"redis-13092.c282.east-us-mz.azure.redns.redis-cloud.com",
        "port":13092,
        "password":"pzSq7pQas9TFd6gMhcypYC70HRdyyvRv",
        "username":"default",
        "decode_responses":True,
        "project_id":"F8AD06364B29E4FFA3AEC9A34E706A47",
        "redis_prefix":"mstr_test"
}}


In [ ]:
# Simple usage example
# Get all objects starting from specific IDs

redis_con_d=redis_env_d["mstr_dev"]
i_redis_bi_analysis = redis_bi_analysis( 
    host=redis_con_d["host"],
    port=redis_con_d["port"],
    password=redis_con_d["password"],
    username=redis_con_d["username"],
    decode_responses=redis_con_d["decode_responses"]
)
i_fetch_it_all=fetch_it_all(i_redis_bi_analysis)

root_object_id_l = ["mstr_test:FILTER:1ACEE9A511D3E95BC000B3B2D86C964F", "mstr_test:DOCUMENT_DEFINITION:98EB31B54122FFB738E6E08A2F29421A"]
all_objects = i_fetch_it_all.fetch_all_objects_recursively(root_object_id_l)

print(f"Found {len(all_objects)} total objects in hierarchy")
for obj in all_objects:
    print(f"Key: {obj['key']}")


Found 200 total objects in hierarchy
Key: mstr_test:FILTER:1ACEE9A511D3E95BC000B3B2D86C964F
Key: mstr_test:DOCUMENT_DEFINITION:98EB31B54122FFB738E6E08A2F29421A
Key: mstr_test:ATTRIBUTE:8D679D3C11D3E4981000E787EC6DE8A4
Key: mstr_test:METRIC:4C05177011D3E877C000B3B2D86C964F
Key: mstr_test:REPORT_DEFINITION:C060EF484E1352B26D945CBAF191F2B8
Key: mstr_test:ATTRIBUTE:54BABECD11D59D57C000B28A4CC5F24F
Key: mstr_test:ATTRIBUTE:8D679D3511D3E4981000E787EC6DE8A4
Key: mstr_test:ATTRIBUTE:8D679D3711D3E4981000E787EC6DE8A4
Key: mstr_test:METRIC:632539C14B768DF229839BA100C92474
Key: mstr_test:METRIC:7FD5B69611D5AC76C000D98A4CC5F24F
Key: mstr_test:ATTRIBUTE:56225F324F17B61AEBD88CA256946C96
Key: mstr_test:ATTRIBUTE:6E069E4C11D3E4E41000E887EC6DE8A4
Key: mstr_test:ATTRIBUTE:8D679D3911D3E4981000E787EC6DE8A4
Key: mstr_test:ATTRIBUTE:8D679D3A11D3E4981000E787EC6DE8A4
Key: mstr_test:ATTRIBUTE:5F0132EE456A636FC8D6E7A59A097AA2
Key: mstr_test:ATTRIBUTE:4151F1304F00A51EB13BCF84283396BE
Key: mstr_test:ATTRIBUTE:54BA

In [ ]:
for o in all_objects:
    if o["definition"]==None:
        print(o["key"])


In [ ]:
all_objects

In [ ]:
class x_fetch_it_all:

    def __init__(self, i_redis_bi_analysis):
        self.i_redis_bi_analysis=i_redis_bi_analysis
    
    def fetch_all_objects_recursively(self, root_object_keys, visited_objects=None):
        """
        Recursively fetch all object definitions from Redis based on child references
        
        Args:

            root_object_keys: List of initial object keys to start with
            visited_objects: Set to track already processed objects (prevents infinite loops)
            
        Returns:
            List of all object definitions (dictionaries)
        """
        if visited_objects is None:
            visited_objects = set()
        
        all_objects = []
        objects_to_process = list(root_object_keys)
        
        while objects_to_process:
            current_key = objects_to_process.pop(0)
            
            # Skip if already processed
            if current_key in visited_objects:
                continue
                
            visited_objects.add(current_key)
            
            try:
                # Fetch object definition from Redis
                obj_def = self.i_redis_bi_analysis.fetch_key_value(current_key)["value"]
                                   
                # Add current object to results
                all_objects.append({
                    'key': current_key,
                    'definition': obj_def
                })
                
                # Extract child object references
                child_keys = self.extract_child_object_keys(obj_def)
                
                # Add child keys to processing queue
                for child_key in child_keys:
                    if child_key not in visited_objects:
                        objects_to_process.append(child_key)
                        
            except Exception as e:
                print(f"Error processing object {current_key}: {e}")
                continue
        
        return all_objects

    def extract_child_object_keys(self, obj_def):
        """
        Extract child object keys from the object definition
        
        Args:
            obj_def: Object definition dictionary
            
        Returns:
            List of child object keys
        """
        child_keys = []
        
        try:
            # Navigate to child_obj_d_l.prefix_child_d_l
            if 'child_obj_d_l' in obj_def:
                child_obj_d_l = obj_def['child_obj_d_l']
                
                if isinstance(child_obj_d_l, dict) and 'prefix_child_d_l' in child_obj_d_l:
                    prefix_child_d_l = child_obj_d_l['prefix_child_d_l']
                    
                    for child_ref in prefix_child_d_l:
                        child_keys.append(child_ref["redis_key"])
                                
        except Exception as e:
            print(f"Error extracting child keys: {e}")
        
        return child_keys

    def fetch_complete_object_hierarchy(self,  root_object_id_l, env_prefix):
        """
        Main function to fetch complete object hierarchy from Redis
        
        Args:
     
            initial_object_ids: List of object IDs to start with
            env_prefix: Environment prefix for Redis keys
            
        Returns:
            List of all related object definitions
        """
       
        # Fetch all objects recursively
        all_objects = self.fetch_all_objects_recursively(

            root_object_keys=root_object_id_l
        )
        
        return all_objects


    # Usage example with your existing Redis setup
    def get_all_related_objects(self,root_object_id_l, env_prefix="mstr_dev"):
        """
        Get all related objects for a list of object IDs
        
        Args:
            object_ids: List of object IDs to start with
            env_prefix: Environment prefix for Redis keys
            
        Returns:
            List of all related object definitions
        """
        # Use your existing Redis connection
        
        # Fetch complete hierarchy
        all_objects = self.fetch_complete_object_hierarchy(
            
            root_object_id_l=root_object_id_l,
            env_prefix=env_prefix
        )
        
        return all_objects





In [ ]:
len(all_objects)